# Applications of Streaming Spanning Tree Algorithm to Segmentation

In this notebook we show three possible applications of Streaming Spanning Tree Algorithm, that we introduced in the previous notebook, to Segmentation. In particular we show how to use it to
- compute $\lambda$-quasi-flat-zones,
- compute a marker-based segmentation,
- compute the $(\alpha, \omega)$-constrained connectivity.

Thanks to the fact that our algorithm splits the minimum spanning tree obtained at time $t$ between a stable and an unstable part, is possible to start treating stable edges to obtain at each step a partial segmentation of the complete image. This avoid to load all the graph in memory and allows also to treat bigger images.

We have written a method for each of the cases listed above. Each of them, at each iteration $t$, yields a partial segmentation coded as a vector $\sigma_t$ of shape $2 \times m_t$. At the first row of vector $\sigma_t$, we list the ids of the nodes/pixels in the image for which we can assign a label in the segmentation, whilst in the second rows we put the labels assigned to nodes in the first row.

In [ ]:
import numpy as np
from scipy.misc import ascent
from matplotlib import pyplot as plt

import loadlib

from SST.utils import *
from SST.streaming.streaming_generators import HorizontalStreaming
from SST.applications.segmentation.streaming_segmentation import quasi_flat_zone_streaming

plt.rcParams['figure.figsize'] = (12.0, 12.0)
%load_ext autoreload

%matplotlib inline

In [ ]:
%autoreload 2

### $\lambda$-quasi-flat-zones

In the following we show how to compute the $\lambda$-quasi-flat-zones of a streaming image. To do so, we implemented a method called ```quasi_flat_zone_streaming```, that at each time $t=0,\ldots, T$ it returns a vector of shape $2\times m_t$, containing at the first row the ids of the nodes/pixels for which we can assing a label, and at the second row the label assigned. So at each step this method yields a partial segmentation of the complete image.  

In [ ]:
# loading image
im = ascent()
img = im[300:400,:500]

plt.imshow(img)
plt.show()

In [ ]:
def plot_segmentation(labels, img_shape, order='F'):

    nr, nc = img_shape

    seg = np.zeros(nr*nc)

    seg[labels[0]] = labels[1]

    plt.imshow(seg.reshape(img_shape, order=order))
    plt.show()

In [ ]:
# creating streaming
gen = HorizontalStreaming(img)
stream = gen.generate_stream(block_shape=(100,100))

threshold = 5
for n, (labels, i) in enumerate(quasi_flat_zone_streaming(stream, threshold, return_img=True)):
    print("Number of new stable labels at iteration {} is {}".format(n, labels.shape[1]))
    if n > 0:
        i = stick_two_images(old_i,i,num_overlapping=1,direction='H')
        labels = np.concatenate((old_labels, labels), axis=1)
    img_shape = i.shape[:2]
    plot_segmentation(labels, img_shape, order='F')
    old_i = i
    old_labels = labels

### Marker based segmentation by MST

In the following we show an example of marker based segmentation by MST. We do so using a method called ```marker_flooding_streaming``` 

In [ ]:
from SST.applications.segmentation.streaming_segmentation import marker_flooding_streaming

In [ ]:
# loading image
im = ascent()
img = im[300:400,:500]

# image shape
nr, nc = img.shape[:2]

plt.imshow(img)
plt.show()

In [ ]:
# generating random markers
num_markers = 100
markers = np.random.randint(0 , nr*nc, size=num_markers)
markers.sort()

In [ ]:
# instatiating image generator
gen = HorizontalStreaming(img)
stream = gen.generate_stream(block_shape=(100,100), markers=markers)

for n, (labels, i) in enumerate(marker_flooding_streaming(stream, return_img=True)):
    print("Number of new stable nodes at iteration {} is {}".format(n, labels.shape[1]))
    if n > 0:
        i = stick_two_images(old_i,i,num_overlapping=1,direction='H')
        labels = np.concatenate((old_labels, labels), axis=1)
    img_shape = i.shape[:2]
    plot_segmentation(labels, img_shape, order='F')
    old_i = i
    old_labels = labels

### $(\alpha,\omega)$-constrained connectivity

In the following we show an example of $(\alpha,\omega)$-constrained connectivity. We do so using a method called ```alpha_omega_cc_streaming``` 

In [ ]:
from SST.applications.segmentation.streaming_segmentation import alpha_omega_cc_streaming

In [ ]:
# loading image
im = ascent()
img = im[300:400,:500]

# image shape
nr, nc = img.shape[:2]

plt.imshow(img)
plt.show()

In [ ]:
gen = HorizontalStreaming(img)
stream = gen.generate_stream(block_shape=(100,100), return_map=True)

alpha = 10
omega = 50

for n, (labels, i) in enumerate(alpha_omega_cc_streaming(stream, alpha=alpha, omega=omega, return_img=True)):
    print("Number of new stable nodes at iteration {} is {}".format(n, labels.shape[1]))
    if n > 0:
        i = stick_two_images(old_i,i,num_overlapping=1,direction='H')
        labels = np.concatenate((old_labels, labels), axis=1)
    img_shape = i.shape[:2]
    plot_segmentation(labels, img_shape, order='F')
    old_i = i
    old_labels = labels